In [1]:
import gym
import numpy as np

In [2]:
env = gym.make("MountainCar-v0")


C:\Users\zhou\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\zhou\anaconda3\lib\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\zhou\anaconda3\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
print(env.observation_space)
print(env.action_space.n)

Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
3


In [4]:
obs_size = [20] * len(env.observation_space.high)
print(obs_size)
win_size = (env.observation_space.high - env.observation_space.low)/obs_size
print(win_size)

[20, 20]
[0.09  0.007]


In [5]:
Q_table_size = obs_size + [env.action_space.n]
print(Q_table_size)

[20, 20, 3]


In [6]:
Q_table = np.random.uniform(-2,0,size=Q_table_size)
print(Q_table)

[[[-1.44817588 -1.65762658 -1.97987974]
  [-0.68799843 -0.75398353 -1.85241015]
  [-1.44417487 -0.92307564 -0.66589347]
  ...
  [-0.49688518 -0.02577782 -1.76252594]
  [-0.49895573 -0.94301957 -0.26326262]
  [-0.58182846 -0.37953771 -0.76317331]]

 [[-0.05309051 -1.79297075 -1.15261765]
  [-1.3058423  -1.7245421  -1.08284638]
  [-1.90605046 -0.27362126 -0.19985229]
  ...
  [-1.89277396 -1.94755666 -1.34958696]
  [-1.04954417 -0.67160575 -0.29472862]
  [-0.8028641  -0.44696634 -1.49379459]]

 [[-1.23517305 -1.46551573 -1.95437716]
  [-1.08964592 -1.52717649 -0.22305184]
  [-0.56529975 -0.98834358 -0.25589597]
  ...
  [-1.87139552 -1.48725262 -0.87135485]
  [-1.41108071 -1.83294622 -0.10865935]
  [-1.33340185 -0.32373763 -0.42722431]]

 ...

 [[-1.16757227 -1.96952321 -1.32062116]
  [-0.13367366 -0.51684523 -0.51168884]
  [-1.17236173 -0.37530242 -0.73507331]
  ...
  [-0.25618645 -0.62097886 -1.37662214]
  [-1.28250766 -1.6713607  -0.61091369]
  [-0.04656523 -1.63939204 -0.28458722]]

 [

In [7]:
def get_qtable_position(status):
   position = (status - env.observation_space.low)/win_size
   return tuple(position.astype(np.int))

In [8]:
position = get_qtable_position(env.reset())
np.argmax(Q_table[position])

<ipython-input-7-0eba0c58d31c>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return tuple(position.astype(np.int))


0

In [9]:
EPISODES = 2000
DISCOUNT = 0.95
LEARNING_RATE = 0.1
epislon = 0.5
END_EPISODE = EPISODES
EPI_DECADE = 0.5/END_EPISODE

STATUS_EVERY = 100
epi_status = {'ep':[],'avg':[],'max':[],'min':[]}
env.reset()
rewards_status = []
for episodes in range(EPISODES):
   env.render()
   observation = env.reset()
   position = get_qtable_position(observation)
   done = False
   epi_reward = 0
   while not done:
      if np.random.random()>epislon:
         action = np.argmax(Q_table[position])
      else:
         action = env.action_space.sample()# User-defined policy function
      observation, reward, done, info = env.step(action)
      if not done:
         q_current = Q_table[position+(action,)]
         new_position = get_qtable_position(observation)
         q_future_max = np.max(Q_table[new_position])
         #q_current + y * q_future_max
         Q_table[position+(action,)] = (1-LEARNING_RATE)*q_current + LEARNING_RATE*(reward + DISCOUNT*q_future_max)
      if observation[0]> env.goal_position:
         print('we make it at {}'.format(episodes))
         Q_table[position+(action,)] = 0
      position = new_position
      epi_reward += reward
   rewards_status.append(epi_reward)
   epislon -= EPI_DECADE
env.close()


C:\Users\zhou\anaconda3\lib\site-packages\gym\core.py:57: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
<ipython-input-7-0eba0c58d31c>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/r

we make it at 376
we make it at 483
we make it at 487
we make it at 622
we make it at 636
we make it at 656
we make it at 657
we make it at 661
we make it at 662
we make it at 664
we make it at 665
we make it at 667
we make it at 670
we make it at 674
we make it at 711
we make it at 785
we make it at 787
we make it at 820
we make it at 830
we make it at 832
we make it at 837
we make it at 838
we make it at 839
we make it at 842
we make it at 843
we make it at 861
we make it at 862
we make it at 863
we make it at 881
we make it at 939
we make it at 968
we make it at 977
we make it at 979
we make it at 985
we make it at 986
we make it at 987
we make it at 988
we make it at 989
we make it at 990
we make it at 992
we make it at 993
we make it at 994
we make it at 996
we make it at 997
we make it at 998
we make it at 1000
we make it at 1001
we make it at 1003
we make it at 1004
we make it at 1005
we make it at 1006
we make it at 1009
we make it at 1010
we make it at 1012
we make it at 1014
